In [6]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats

# Loading in data

In [7]:
dataPath = 'Data/2016-2019(One station)/shair-8781-1-6-1.csv'
airQualityData=pd.read_csv(dataPath, header=14,sep=';').rename(columns={'Start':'Start','Slut':'Stop'})
airQualityData.rename(columns = lambda x: re.sub('NOX.*','NOX',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('PM10.*','PM10',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('PM2.5.*','PM2_5',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('NO2.*','NO2',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('O3.*','O3',x), inplace = True)
airQualityData.rename(columns = lambda x: re.sub('Black Carbon.*','Black Carbon',x), inplace = True)
airQualityData['Start'] = pd.to_datetime(airQualityData['Start'])
airQualityData= airQualityData.drop('Stop',axis=1)
one_feature=airQualityData.drop(columns=['Black Carbon','NO2','NOX','O3','PM2_5'])#
one_feature.head(5)

,Start,PM10
0,2015-01-01 00:00:00,28.4
1,2015-01-01 01:00:00,8.5
2,2015-01-01 02:00:00,8.7
3,2015-01-01 03:00:00,10.5
4,2015-01-01 04:00:00,12.1


##### Scaling

In [8]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
scaled_down=one_feature.copy()
scaled_down['PM10']=sc.fit_transform(scaled_down['PM10'].values.reshape(-1, 1))

In [9]:
train=scaled_down[(scaled_down['Start']<= "2018-12-31 23:00:00")]
test=scaled_down[(scaled_down['Start'] >= "2019-01-01 00:00:00")]

NameError: name 'X' is not defined

In [10]:
from keras.preprocessing.sequence import TimeseriesGenerator
train_x = train.copy()
train_x=train_x.fillna(0)
train_x=train_x.drop('Start', axis = 1)
temp_X_train = train_x
temp_X_train = temp_X_train.values
temp_X_train = temp_X_train.reshape(len(temp_X_train))
n_input = 5 #how many samples/rows/timesteps to look in the past in order to forecast the next sample
n_features= 1 # how many predictors/Xs/features we have to predict y
b_size = 32 # Number of timeseries samples in each batch
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(32, 5)


In [13]:
def main(dataframe):
    for i in range(1,10,1):
        percentage_nb= int(len(dataframe)*(i/10))
        current_train=dataframe[:,percentage_nb]
        print(current_train)

In [14]:
generator_train = TimeseriesGenerator(temp_X_train, temp_X_train, length=n_input, batch_size=b_size)
print(generator_train[0][0].shape)

TypeError: '(slice(None, None, None), 3506)' is an invalid key